In [1]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

%load_ext autoreload
%autoreload 2

from utils import helper, config, rayer, kaggle_dataset_helper

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from ml.models.ensemble import Ensemble
from ml.xai.model.explainable import Explainable

from sklearn.metrics import make_scorer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common

import ray


import time
import pandas as pd
import pyarrow.fs

In [2]:
# rayer.get_global_cluster()
rayer.get_local_cluster()

<module 'ray' from '/home/vagrant/miniconda3/envs/automl/lib/python3.8/site-packages/ray/__init__.py'>

In [3]:
def use_tokamat_ds():
    df = helper.get_tokamat_dataset()
    df = df.reset_index()

    df = common.label_encode(df)
    df = df.fillna(-1)

    potential_targets = ['WTOT', 'WTH', 'PLTH']
    y = df[potential_targets[0]]

    X = df[df.columns[~df.columns.isin(potential_targets)]]
    X = X.drop(['TOK_ID', 'LCUPDATE', 'DATE', 'NEL', 'ENBI'], axis = 1)

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    return train_test_split(X, y, test_size=0.33)


def use_covid_ds():
    df_X, df_y = helper.get_covid_dataset()
    df_X = df_X.drop(['location'], axis = 1)

    return train_test_split(df_X, df_y, test_size=0.33)


def use_transaction_predictions_ds():
    ds_train, ds_test = kaggle_dataset_helper.get_transaction_predictions_dataset()
    ds_train = common.label_encode(ds_train)
    ds_test = common.label_encode(ds_test)

    ds_train = ds_train.fillna(-1)
    ds_test = ds_test.fillna(-1)

    df_X = ds_train.loc[:, ds_train.columns != 'target']
    df_y = ds_train['target']

    return train_test_split(df_X, df_y, test_size=0.33, random_state=config.rand_state)



def use_house_pricing_ds():
    ds_train, ds_test = kaggle_dataset_helper.get_house_prices_dataset()
    ds_train = common.label_encode(ds_train)
    ds_test = common.label_encode(ds_test)

    ds_train = ds_train.fillna(-1)
    ds_test = ds_test.fillna(-1)


    df_X = ds_train.loc[:, ds_train.columns != 'SalePrice']
    df_y = ds_train['SalePrice']

    return train_test_split(df_X, df_y, test_size=0.33, random_state=config.rand_state)


In [4]:
X_train, X_test, y_train, y_test = use_tokamat_ds()

/mnt/c/Users/rwmas/GitHub/xai/python-asd/xai_0.2.0/notebooks/relevance/../../utils


In [5]:
r2_scoring = make_scorer(score_func=r2_score, greater_is_better=False)

In [ ]:
ens_mdl = Ensemble(   
                                xgb_objective='binary:logistic',  # ["reg:squarederror", "count:poisson", "binary:logistic",  "binary:hinge" ]
                                lgbm_objective='binary',    # https://lightgbm.readthedocs.io/en/latest/Parameters.html
                                pred_class='classification',
                                score_func=None,
                                metric_func=None,
                                list_base_models=[],
                                n_trials=10,          ### common param
                                epochs=15,             ### ANN param
                                boosted_round=10,      ### boosting tree param
                                max_depth=30,          ### boosting tree param
                                max_n_estimators=1500, ### rf param
                                n_estimators=30,       ### bagging param, must be > 10 
                                n_neighbors=30,        ### knn param, must be > 5

                                ensemble_n_estimators=30,  ###  must be > 10
                                ensemble_n_trials=10,
                                timeout=None
                 )

In [6]:
# list_base_models = ['briskbagging', 'briskknn', 'briskxgboost', 'slugxgboost', 'sluglgbm','slugrf']

ens_mdl = Ensemble(   
                                xgb_objective='count:poisson',  # ["reg:squarederror", "count:poisson", "binary:logistic",  "binary:hinge" ]
                                lgbm_objective='poisson',    # https://lightgbm.readthedocs.io/en/latest/Parameters.html
                                pred_class='regression',
                                score_func=None,
                                metric_func=None,
                                list_base_models=['sluglgbm'],
                                n_trials=10,          ### common param
                                epochs=15,             ### ANN param
                                boosted_round=10,      ### boosting tree param
                                max_depth=30,          ### boosting tree param
                                max_n_estimators=1500, ### rf param
                                n_estimators=30,       ### bagging param, must be > 10 
                                n_neighbors=30,        ### knn param, must be > 5

                                ensemble_n_estimators=30,  ###  must be > 10
                                ensemble_n_trials=10,
                                timeout=None
                 )

In [7]:
ens_mdl.fetch_models(X_train, X_test, y_train, y_test)

2022-12-07 11:49:10,799	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'cv': KFold(n_splits=3, random_state=None, shuffle=False), 'early_stop_type': <EarlyStopping.NO_EARLY_STOP: 7>, 'scoring/score': <function _passthrough_scorer at 0x7f2147f0f430>}
2022-12-07 11:49:10,841	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'cv': KFold(n_splits=3, random_state=None, shuffle=False), 'early_stop_type': <EarlyStopping.NO_EARLY_STOP: 7>, 'scoring/score': <function _passthrough_scorer at 0x7f2147f0f430>}
2022-12-07 11:49:10,886	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'cv': KFold(n_splits=3, random_state=None, shuffle=False), 'early_stop_type': <EarlyStopping.NO_EARLY_STOP: 7>, 'scoring/score': <function _passthrough_scorer at 0x7f2147f0f430>}
2022-12-07 11:49:10,921	INFO tensorboardx.py:267 -- Removed the following hyperparame

(<ray.tune.trainable.util._Trainable object at 0x7f1095078250> pid=15054) [LightGBM] [Warning] lambda_l1 is set=0.0013047486135049219, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0013047486135049219
(<ray.tune.trainable.util._Trainable object at 0x7f1095078250> pid=15054) [LightGBM] [Warning] bagging_fraction is set=4.5622344320629684e-05, subsample=1.0 will be ignored. Current value: bagging_fraction=4.5622344320629684e-05
(<ray.tune.trainable.util._Trainable object at 0x7f1095078250> pid=15054) [LightGBM] [Warning] lambda_l2 is set=0.016452992170058762, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.016452992170058762
(<ray.tune.trainable.util._Trainable object at 0x7f1095078250> pid=15054) [LightGBM] [Warning] learning_rate is set=0.1, eta=0.09449437412561167 will be ignored. Current value: learning_rate=0.1
(<ray.tune.trainable.util._Trainable object at 0x7fea8ebda190> pid=15060) [LightGBM] [Warning] lambda_l1 is set=0.027505887873708206, reg_alpha=0.0 wil

2022-12-07 11:49:10,967	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'cv': KFold(n_splits=3, random_state=None, shuffle=False), 'early_stop_type': <EarlyStopping.NO_EARLY_STOP: 7>, 'scoring/score': <function _passthrough_scorer at 0x7f2147f0f430>}
2022-12-07 11:49:10,994	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'cv': KFold(n_splits=3, random_state=None, shuffle=False), 'early_stop_type': <EarlyStopping.NO_EARLY_STOP: 7>, 'scoring/score': <function _passthrough_scorer at 0x7f2147f0f430>}
2022-12-07 11:49:11,011	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'cv': KFold(n_splits=3, random_state=None, shuffle=False), 'early_stop_type': <EarlyStopping.NO_EARLY_STOP: 7>, 'scoring/score': <function _passthrough_scorer at 0x7f2147f0f430>}
2022-12-07 11:49:11,030	INFO tensorboardx.py:267 -- Removed the following hyperparame

(<ray.tune.trainable.util._Trainable object at 0x7f1095078250> pid=15054) [LightGBM] [Warning] lambda_l1 is set=0.00361663314180572, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00361663314180572
(<ray.tune.trainable.util._Trainable object at 0x7f1095078250> pid=15054) [LightGBM] [Warning] bagging_fraction is set=2.3064443562318316e-05, subsample=1.0 will be ignored. Current value: bagging_fraction=2.3064443562318316e-05
(<ray.tune.trainable.util._Trainable object at 0x7f1095078250> pid=15054) [LightGBM] [Warning] lambda_l2 is set=0.008783119450287348, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.008783119450287348
(<ray.tune.trainable.util._Trainable object at 0x7f1095078250> pid=15054) [LightGBM] [Warning] learning_rate is set=0.1, eta=0.0005074112458039998 will be ignored. Current value: learning_rate=0.1
(<ray.tune.trainable.util._Trainable object at 0x7f1095078250> pid=15054) [LightGBM] [Warning] lambda_l1 is set=0.00361663314180572, reg_alpha=0.0 will b

2022-12-07T11:49:11CET : INFO : ensemble : fetch_models : 215 : Message : base model training completed


[LightGBM] [Warning] lambda_l1 is set=0.08946011299943274, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.08946011299943274
[LightGBM] [Warning] bagging_fraction is set=0.0009640208010365888, subsample=1.0 will be ignored. Current value: bagging_fraction=0.0009640208010365888
[LightGBM] [Warning] lambda_l2 is set=0.01141317044996692, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01141317044996692
[LightGBM] [Warning] learning_rate is set=0.1, eta=5.137945724851099e-06 will be ignored. Current value: learning_rate=0.1


In [8]:
len(ens_mdl.base_models)

1

In [10]:
attr_algos = ['IG', 'SHAP', 'GradientSHAP'] #, 'knockoff']
ex = Explainable(ens_mdl, X_train)

In [11]:
ex.get_attr(attr_algos)

2022-12-07T08:54:24CET : INFO : explainable_tune : get_attr : 153 : Message : attribution methods  ['ig', 'shap', 'gradientshap']
2022-12-07T08:54:24CET : INFO : explainable_tune : get_attr : 156 : Message : calculating variable importance on  slug_lgbm
2022-12-07T08:54:25CET : INFO : explainable_tune : get_attr : 156 : Message : calculating variable importance on  brisk_xgboost
(__get_shapley_ensemble_attr__ pid=657056) ntree_limit is deprecated, use `iteration_range` or model slicing instead.


cols
IP         62.5
index      58.5
TIME_ID    53.5
RGEO       31.5
SHOT       31.0
           ... 
BGASZ       0.0
BGASA       0.0
NELFORM     0.0
DNELDT      0.0
TOK         0.0
Length: 64, dtype: float64